In [1]:
from src.analytics.preprocess import TextNormalizer
import pandas as pd

dataframe = pd.read_csv('data/train_data.csv').drop(columns=['timestamp'])
normalizer = TextNormalizer('data/stopwords.txt')
normalizer.fit()

TextNormalizer(stopwords_filename='data/stopwords.txt')

In [2]:
def concat_columns(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe['text'] = (dataframe['question_1'] + ' '
                         + dataframe['question_2'] + ' '
                         + dataframe['question_3'] + ' '
                         + dataframe['question_4'] + ' '
                         + dataframe['question_5'])
    return dataframe.drop(columns=['question_1',
                                   'question_2',
                                   'question_3',
                                   'question_4',
                                   'question_5'])


dataframe = concat_columns(dataframe)
dataframe

,is_relevant,object,is_positive,text
0,1,0,1,Основы программирования Интерактивный подход к...
1,1,0,1,Основы программирования Подробное объяснение л...
2,1,0,1,Основы программирования Введение в программиро...
3,1,0,1,Основы программирования Понятное изложение осн...
4,1,0,1,Основы программирования Кларность и структура ...
...,...,...,...,...
98,1,2,1,Основы программирования Преподователь раз тако...
99,1,2,1,Новейшие тенденции в IT Очень понравилось изуч...
100,1,2,0,Новейшие тенденции в IT Очень подробный разбор...
101,1,2,0,"Java api браузеров Понравилась тема Да, препод..."


In [3]:
normalizer.transform(dataframe['text'].to_list())

['основа программирование интерактивный подход изучение основа увлекательный увеличить количество практический задание хороший закрепление материал введение разработка net framework',
 'основа программирование подробный объяснение логический оператор javascript помочь условный конструкция предоставить практический задача теория детальный изучение асинхронный программирование javascript',
 'основа программирование введение программирование python простой интуитивно понятный начинать улучшить качество видеоматериал трудный разглядеть код библиотека python анализ данные машинный обучение',
 'основа программирование понятный изложение основный понятие хороший структура материал цикл for while объяснить быстро добавить видеоматериал самостоятельный изучение функциональный программирование',
 'основа программирование кларность структура материал основа javascript затронуть тема цикл функция ввести пример работа массив объект асинхронный код javascript',
 'основа программирование простота объ

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

transformer = Pipeline(steps=[
    ('normalize', TextNormalizer('data/stopwords.txt')),
    ('vectorizer', CountVectorizer(
        ngram_range=(1, 1),
        min_df=1,
        max_df=1
    ))
])
transformer.fit(dataframe['text'].to_list())

Pipeline(steps=[('normalize',
                 TextNormalizer(stopwords_filename='data/stopwords.txt')),
                ('vectorizer', CountVectorizer(max_df=1))])

In [11]:
X = transformer.transform(dataframe['text'].to_list())
y_relevant = dataframe['is_relevant'].to_numpy()
y_object = dataframe['object'].to_numpy()
y_positive = dataframe['is_positive'].to_numpy()

In [12]:
from sklearn.metrics import classification_report


def format_report(relevant_model,
                  object_model,
                  positive_model) -> str:
    return ('relevant classification\n' +
           f'{classification_report(y_relevant, relevant_model.predict(X))}\n' +
           'object classification\n' +
           f'{classification_report(y_object, object_model.predict(X))}\n' +
           'positive classification\n' +
           f'{classification_report(y_object, positive_model.predict(X))}\n')

In [14]:
from sklearn.naive_bayes import BernoulliNB as classifier

clf_relevant = classifier().fit(X, y_relevant)
clf_object = classifier().fit(X, y_object)
clf_positive = classifier().fit(X, y_positive)

print(format_report(clf_relevant, clf_object, clf_positive))

/Users/danila/Documents/Python Projects/AI Hackaton 26.04.2024/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/danila/Documents/Python Projects/AI Hackaton 26.04.2024/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/danila/Documents/Python Projects/AI Hackaton 26.04.2024/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

'relevant classification\n              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        13\n           1       0.87      0.99      0.93        90\n\n    accuracy                           0.86       103\n   macro avg       0.44      0.49      0.46       103\nweighted avg       0.76      0.86      0.81       103\n\nobject classification\n              precision    recall  f1-score   support\n\n           0       0.64      1.00      0.78        38\n           1       1.00      0.40      0.57        30\n           2       1.00      0.91      0.96        35\n\n    accuracy                           0.80       103\n   macro avg       0.88      0.77      0.77       103\nweighted avg       0.87      0.80      0.78       103\n\npositive classification\n              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        38\n           1       0.28      0.93      0.43        30\n           2       0.00      0.0